In [1]:
import csv
import os
import traceback
import numpy as np
import tqdm
import pandas as pd
import multiprocessing as mp
import gc
from datetime import timedelta, date
import uuid

# import rsa_data_summary as rd
# import rsa_data_wim as wim
# import rsa_headers as rh
import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

In [205]:
# individual tables for type 30 and 60 test
# files = tools.getfiles(r'C:\FTP\Syntell\SMEC RSA Files_GP PRM Sites_Dec21toFeb22')
file = r"C:\PQ410\2 - Data In\TIS Data\GDRT\GP_RSA 2014\0038.RSA"
t21_file =r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_1.RSA"
t10_file = r"C:\FTP\Syntell\SMEC RSV Files_GP PRM Sites_Dec21toFeb22_individuals\0006-20211231.RSV"
t10_file2 = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
type60_file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"

In [3]:
# electronic_count_data_type_21_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_21 limit 1",config.ENGINE).columns)
# electronic_count_data_type_30_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_30 limit 1",config.ENGINE).columns)
electronic_count_data_type_60_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_60 limit 1",config.ENGINE).columns)
# electronic_count_data_type_70_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_70 limit 1",config.ENGINE).columns)
# header_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_header limit 1",config.ENGINE).columns)

In [359]:
# df = tools.to_df(t10_file)
# dfa = pd.read_csv(t10_file, header=None) #, sep='\s+', engine='python')
dfa = pd.read_csv(t10_file, header=None, sep='\s+', engine='python')

dfa = dfa[0].str.split("\s+|,\s+|,|;|;\s+", expand=True)

In [158]:
df = pd.DataFrame(dfa.loc[
            (~dfa[0].isin(["H0", "H9", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
            & (
                (dfa[0].isin(["10"]))
                & (~dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
                & (dfa.loc[dfa[0].isin(["10"])][4].astype(int) > 80)
            )
        ]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [360]:
df = pd.DataFrame(dfa.loc[
    (~dfa[0].isin(["H0", "H9", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    & ((
        (dfa[0].isin(["21", "22", "70", "30", "31", "13", "60"]))
        & (dfa[1].isin(["0", "1", "2", "3", "4"]))
        & (dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"])][3].astype(int) > 80)
    ) | (
        (dfa[0].isin(["10"]))
        & (~dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
        & (dfa.loc[dfa[0].isin(["10"])][4].astype(int) > 80)
    ))           
    ]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [362]:
data = df.loc[(df[0] == "10")].dropna(
                axis=1, how="all"
            ).reset_index(drop=True).copy()

In [363]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,10,15,1,0,211201,0000006,3,3,1,10,1,1,82,437,233,3,1
1,10,15,1,0,211201,0000128,2,2,1,10,1,1,85,406,215,3,1
2,10,15,1,0,211201,0000221,4,4,1,10,1,1,72,546,296,1,1
3,10,15,1,0,211201,0000291,1,1,1,10,1,1,69,442,278,2,1
4,10,15,1,0,211201,0000472,2,2,1,10,1,1,73,456,271,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307748,10,15,1,0,211231,2359172,4,4,1,10,1,1,51,471,365,1,2
1307749,10,15,1,0,211231,2359350,2,2,1,10,1,1,101,391,175,3,1
1307750,10,15,1,0,211231,2359458,2,2,1,10,1,1,122,424,155,3,1
1307751,10,15,1,0,211231,2359523,3,3,1,10,1,1,41,373,418,3,1


In [300]:
df = pd.DataFrame(dfa.loc[
    (~dfa[0].isin(["H0", "H9", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    & (
        (dfa[0].isin(["21", "22", "70", "30", "31", "13", "60"]))
        & (dfa[1].isin(["0", "1", "2", "3", "4"]))
        & (dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"])][2].astype(int) > 80)
    )]).head().dropna(axis=1, how="all").reset_index(drop=True).copy()

In [301]:
head_df = pd.DataFrame(dfa.loc[
    (dfa[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    | (
        (dfa[0].isin(["21", "70", "30", "13", "60"]))
        # & (dfa.loc[dfa[2].isin(["21", "70", "30", "13", "60"])][2].astype(int) < 80)
    )
    | (
        (dfa[0].isin(["10"]))
        & (dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
    )
]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [296]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,21,0,071121,1300,15,01,0000,0001,0000,0002,0005,0003,0002,0001,0000,0000,000000,000,000,000,0002,0003
1,21,0,071121,1300,15,02,0002,0002,0001,0004,0002,0003,0003,0002,0000,0000,000172,001,001,001,0002,0004
2,21,0,071121,1315,15,01,0000,0001,0003,0001,0004,0000,0004,0000,0000,0000,000347,001,002,001,0000,0000
3,21,0,071121,1315,15,02,0001,0001,0004,0001,0002,0000,0001,0000,0000,0000,000271,002,001,001,0001,0001
4,21,0,071121,1330,15,01,0001,0000,0001,0006,0002,0002,0002,0000,0000,0000,000212,001,002,000,0001,0001


False

In [303]:
if df.loc[df[0]=="10"].empty:
    duration_min = int(df[4].at[0])
    date_col = 2
    time_col = 3
    add_day = 1
    date_col_name = "end_date"
    time_col_name = "end_time"
    typ = "sum"
else:
    date_col = 4
    time_col = 5
    duration_min = 0
    add_day = 0
    date_col_name = "departure_date"
    time_col_name = "departure_time"
    typ = "indv"
date_format = "%Y-%m-%d" # 2021-12-01
time_format = "%H:%M:%S.%f" # 00:00:00.00
    
date_length = len(df[date_col].at[0])

df[time_col] = df[time_col].astype(str)

df[time_col] = df[time_col].str.pad(width=7,side='right',fillchar="0")
df[time_col].loc[df[time_col].str[:2] == '24'] = ('0').zfill(7)

print("date_col:" + str(date_col))
print("time_col:" + str(time_col))

print(date_length)
print(df[date_col].head())

if date_length == 6:
    decade = int(df[date_col].at[0][:2])
    if decade < 50:
        century = str(date.today())[:2]
    else:
        century = int(str(date.today())[:2])-1
    df[date_col] = str(century) + df[date_col]
elif date_length == 8:
    pass
else:
    raise Exception("DATA Date length abnormal. length = "+str(date_length))

if typ == "indv":
    df[date_col] = pd.to_datetime(df[date_col], format="%Y%m%d").dt.strftime(date_format)
else:
    df[date_col] = df[date_col].apply(lambda x: pd.to_datetime(
        x, format="%Y%m%d").date() + timedelta(days=add_day)
        if x[time_col] in ['0'.zfill(7),'24'.ljust(7,'0')] 
        else pd.to_datetime(x, format="%Y%m%d").date())

df[time_col] = pd.to_datetime(df[time_col], format="%H%M%S%f").dt.strftime(time_format)
# df[time_col] = df[time_col].str[:11].astype(str)
print((df[date_col].astype(str)+df[time_col].astype(str)).head())


df['end_datetime'] = pd.to_datetime((df[date_col].astype(str)+df[time_col].str[:10].astype(str)), 
    format=date_format+time_format)

df['start_datetime'] = pd.to_datetime((df[date_col].astype(str)+df[time_col].str[:10].astype(str)), 
    format=date_format+time_format) - timedelta(minutes=duration_min)

df.rename(columns={
    date_col: date_col_name,
    time_col: time_col_name
},inplace = True)

KeyError: 2

ValueError: Could not convert object to NumPy timedelta

In [313]:
if not dfa.empty:
    lanes = dfa.loc[
        dfa[0]=="L1"
        ].dropna(axis=1).drop_duplicates().reset_index(drop=True).copy()
    lanes = lanes.drop([17, 18, 19, 20, 21, 22, 23, 24, 25], axis=1, errors='ignore')
    if lanes.shape[1] == 5:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number"
        },inplace = True)
    elif lanes.shape[1] == 11:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number",
            5 : "traffic_stream_lane_position",
            6 : "reverse_direction_lane_number",
            7 : "vehicle_code",
            8 : "time_code",
            9 : "length_code",
            10 : "speed_code"
        },inplace = True)
    elif lanes.shape[1] == 17:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number",
            5 : "traffic_stream_lane_position",
            6 : "reverse_direction_lane_number",
            7 : "vehicle_code",
            8 : "time_code",
            9 : "length_code",
            10 : "speed_code",
            11 : "occupancy_time_code",
            12 : "vehicle_following_code",
            13 : "trailer_code",
            14 : "axle_code",
            15 : "mass_code",
            16 : "tyre_type_code"
        },inplace = True)
    else:
        lanes.rename(columns={
            1 : "lane_number",
            2 : "direction_code",
            3 : "lane_type_code",
            4 : "traffic_stream_number"
        },inplace = True)
    lanes[lanes.select_dtypes(include=['object']).columns] = lanes[
        lanes.select_dtypes(include=['object']).columns].apply(
            pd.to_numeric, axis=1, errors='ignore')
    lanes["site_name"] = 'site_id'
else:
    pass


In [337]:
lanes

,0,lane_number,direction_code,lane_type_code,traffic_stream_number,traffic_stream_lane_position,reverse_direction_lane_number,vehicle_code,time_code,length_code,speed_code,site_name
0,L1,01,0,1,02,0,0,0,0,0,0,site_id
1,L1,02,4,1,01,0,0,0,0,0,0,site_id
2,L1,01,0,1,02,0,1,1,1,0,0,site_id
3,L1,02,4,1,01,0,1,1,1,0,0,site_id


In [166]:
df.head()

,0,1,2,3,departure_date,departure_date,6,7,8,9,10,11,12,13,14,15,16,end_datetime,start_datetime
0,10,15,1,0000000,2021-12-01,00:00:00.600000,3,3,1,10,1,1,82,437,233,3,1,2021-12-01 00:00:00.600,2021-12-01 00:00:00.600
1,10,15,1,0000000,2021-12-01,00:00:12.800000,2,2,1,10,1,1,85,406,215,3,1,2021-12-01 00:00:12.800,2021-12-01 00:00:12.800
2,10,15,1,0000000,2021-12-01,00:00:22.100000,4,4,1,10,1,1,72,546,296,1,1,2021-12-01 00:00:22.100,2021-12-01 00:00:22.100
3,10,15,1,0000000,2021-12-01,00:00:29.100000,1,1,1,10,1,1,69,442,278,2,1,2021-12-01 00:00:29.100,2021-12-01 00:00:29.100
4,10,15,1,0000000,2021-12-01,00:00:47.200000,2,2,1,10,1,1,73,456,271,3,1,2021-12-01 00:00:47.200,2021-12-01 00:00:47.200


In [354]:
try:
    dir_1 = lanes["direction_code"].astype(int).min()
    dir_2 = lanes["direction_code"].astype(int).max()
except (TypeError,ValueError):
    dir_1 = 0
    dir_2 = 4
    
if df.loc[df[0]=="10"].empty:
    lane_col = 5
else:
    lane_col = 6


df['direction'] = df[lane_col].astype(int)
df['compass_heading'] = df[lane_col].astype(int)
df['direction_code'] = df[lane_col].astype(int)
df['direction'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])
    )] = 'P'
df['direction'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_2])
    )] = 'N'
df['compass_heading'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])
    )] = str(dir_1)
df['compass_heading'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_2])
    )] = str(dir_2)
df['direction_code'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_2])
    )] = str(dir_2)
df['direction_code'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])
    )] = str(dir_1)


In [355]:
df

,0,1,end_date,end_time,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,end_datetime,start_datetime,direction,compass_heading,direction_code
0,21,0,2007-11-21,13:00:00.000000,15,01,0000,0001,0000,0002,0005,0003,0002,0001,0000,0000,000000,000,000,000,0002,0003,2007-11-21 13:00:00,2007-11-21 12:45:00,P,0,0
1,21,0,2007-11-21,13:00:00.000000,15,02,0002,0002,0001,0004,0002,0003,0003,0002,0000,0000,000172,001,001,001,0002,0004,2007-11-21 13:00:00,2007-11-21 12:45:00,N,4,4
2,21,0,2007-11-21,13:15:00.000000,15,01,0000,0001,0003,0001,0004,0000,0004,0000,0000,0000,000347,001,002,001,0000,0000,2007-11-21 13:15:00,2007-11-21 13:00:00,P,0,0
3,21,0,2007-11-21,13:15:00.000000,15,02,0001,0001,0004,0001,0002,0000,0001,0000,0000,0000,000271,002,001,001,0001,0001,2007-11-21 13:15:00,2007-11-21 13:00:00,N,4,4
4,21,0,2007-11-21,13:30:00.000000,15,01,0001,0000,0001,0006,0002,0002,0002,0000,0000,0000,000212,001,002,000,0001,0001,2007-11-21 13:30:00,2007-11-21 13:15:00,P,0,0


In [347]:
lanes

,0,lane_number,direction_code,lane_type_code,traffic_stream_number,traffic_stream_lane_position,reverse_direction_lane_number,vehicle_code,time_code,length_code,speed_code,site_name
0,L1,01,0,1,02,0,0,0,0,0,0,site_id
1,L1,02,4,1,01,0,0,0,0,0,0,site_id
2,L1,01,0,1,02,0,1,1,1,0,0,site_id
3,L1,02,4,1,01,0,1,1,1,0,0,site_id


In [353]:
list(lanes['lane_number'].astype(int).loc[lanes['direction_code'].astype(int) == dir_1])

[1, 1]

In [350]:
df['direction_code'].loc[df[lane_col].astype(int).isin(
    list(lanes['lane_number'].astype(int).loc[lanes['direction_code'] == dir_1])
    )]

Series([], Name: direction_code, dtype: object)